In [ ]:
%%capture
%pip install accelerate peft bitsandbytes trl peft tensorboard transformers==4.38.2

In [ ]:
!pip install git+https://github.com/huggingface/huggingface_hub.git@main
from huggingface_hub import login

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig,HfArgumentParser,TrainingArguments,pipeline, logging
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
import os,torch
from datasets import load_dataset
from trl import SFTTrainer

In [ ]:
login(token=os.environ['HF_TOKEN'])

In [ ]:
model_name = 'mistralai/Mistral-7B-Instruct-v0.2'
dataset_name = "final_data_chat.json" # The instruction dataset to use
new_model = 'your-new-model-name' # Fine-tuned model name

In [ ]:
# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, 'float16')

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False,
)

In [ ]:
# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map='auto',
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
)

model.config.use_cache = False
model.config.pretraining_tp = 1

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training
tokenizer.add_bos_token, tokenizer.add_eos_token

In [ ]:
model = prepare_model_for_kbit_training(model)
# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=64,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj","gate_proj"]
)
model = get_peft_model(model, peft_config)

In [ ]:
def generate_and_tokenize_prompt(prompt):
    return tokenizer(prompt['text'])

In [ ]:
data = load_dataset("json", data_files=dataset_name)

In [ ]:
dataset = data['train'].map(generate_and_tokenize_prompt)

In [ ]:
# Set training parameters
training_arguments = TrainingArguments(
    output_dir='./results',
    num_train_epochs=1,
    per_device_train_batch_size=8,
    gradient_accumulation_steps=1,
    optim='paged_adamw_32bit',
    save_steps=100000,
    logging_steps=250,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=True,
    max_grad_norm=0.3,
    max_steps=1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type='constant',
)

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=2048,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=False,
)

In [ ]:

# Train model
trainer.train()

# Save trained model
trainer.model.save_pretrained(new_model)